In [37]:
# load dependencies
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [38]:
# Load data and select trainging hour and test hour
trainhour = 5
testhour = 7
df = pd.read_csv('series.csv')
cols = ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10']
cols10 = ['31', '32', '33', '34', '35', '36', '37', '38', '39', '40']
df['begin'] = df[cols].sum(axis=1)
df['sum10'] = df[cols10].sum(axis=1)
data = df.drop(df[(df['hour'] != trainhour)].index)
testdata = df.drop(df[(df['hour'] != testhour)].index)
del df

In [39]:
# Categorize the 'sum' column the number of PIPs over selected range
def setlabels(x):
    if x < -10:
        return -1
    elif x > 10:
        return 1
    else:
        return 0

data["labels"] = data["sum10"].apply(setlabels)

In [40]:
X_data = data.iloc[:,18:38]
X_data['begin'] = data['begin']
X_data.head()

,11,12,13,14,15,16,17,18,19,20,...,22,23,24,25,26,27,28,29,30,begin
716,0.4,0.8,-1.1,0.1,0.2,-0.5,-1.1,-2.0,0.0,-1.7,...,-0.7,-1.9,-0.2,1.2,1.6,1.5,1.3,0.1,1.7,0.6
717,0.8,-1.1,0.1,0.2,-0.5,-1.1,-2.0,0.0,-1.7,1.5,...,-1.9,-0.2,1.2,1.6,1.5,1.3,0.1,1.7,-0.4,-2.2
718,-1.1,0.1,0.2,-0.5,-1.1,-2.0,0.0,-1.7,1.5,-0.7,...,-0.2,1.2,1.6,1.5,1.3,0.1,1.7,-0.4,2.7,-6.4
719,0.1,0.2,-0.5,-1.1,-2.0,0.0,-1.7,1.5,-0.7,-1.9,...,1.2,1.6,1.5,1.3,0.1,1.7,-0.4,2.7,2.2,-8.5
720,0.2,-0.5,-1.1,-2.0,0.0,-1.7,1.5,-0.7,-1.9,-0.2,...,1.6,1.5,1.3,0.1,1.7,-0.4,2.7,2.2,-1.0,-3.8


In [41]:
# Get X, y for model
X = X_data.to_numpy()

from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical
y = data['labels']
label_encoder = LabelEncoder()
label_encoder.fit(y)
y_encoded = label_encoder.transform(y)
y = to_categorical(y_encoded)

In [42]:
# Split data into training and test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y)

In [43]:
# Create model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
model = Sequential()
model.add(Dense(units=100, activation='relu', input_dim=X_train.shape[1]))
model.add(Dense(units=100, activation='relu'))
model.add(Dense(units=y_train.shape[1], activation='softmax'))
model.summary()

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_9 (Dense)              (None, 100)               2200      
_________________________________________________________________
dense_10 (Dense)             (None, 100)               10100     
_________________________________________________________________
dense_11 (Dense)             (None, 3)                 303       
Total params: 12,603
Trainable params: 12,603
Non-trainable params: 0
_________________________________________________________________


In [44]:
# Fit model
model.fit(X_train, y_train, epochs=5, shuffle=True, verbose=0)

In [45]:
# Model loss and Accuray
model_loss, model_accuracy = model.evaluate(X_train, y_train, verbose=0)
print(f"Normal Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")
model_loss, model_accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"Normal Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")

Normal Neural Network - Loss: 0.16126678166914202, Accuracy: 0.9602643251419067
Normal Neural Network - Loss: 0.21309926458293943, Accuracy: 0.9562307000160217


In [46]:
formodel = testdata.iloc[:,18:38]
formodel['begin'] = testdata['begin']
formodel = formodel.to_numpy()
formodel.shape

(15526, 21)

In [47]:
# Pull out validation data and make predicts
testdata['class'] = label_encoder.inverse_transform(model.predict_classes(formodel))
testdata = testdata.drop(testdata[(testdata['class'] == 0)].index)

In [48]:
totalrows = testdata.shape[0]
totalcols = testdata.shape[1]
profit = []
for i in range(totalrows):
    temp = 0
    for j in range(10):
        temp += testdata.iloc[i,28+j]
        if (testdata.iloc[i,totalcols-1] == -1):
            if(temp < 2):
              break  
        else:
            if(temp < -2):
              break
        
    if (testdata.iloc[i,totalcols-1] == -1):
        temp = -temp
    profit.append(temp)

print("Est Profit: " + str(sum(profit)))

Est Profit: 522.4
